In [ ]:
# Colab: install required libs
!pip install -q mlxtend imbalanced-learn shap

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import KBinsDiscretizer
from mlxtend.frequent_patterns import apriori, association_rules
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, precision_recall_curve, auc
from imblearn.over_sampling import SMOTE
import joblib
import shap
import warnings
warnings.filterwarnings('ignore')

from google.colab import drive
drive.mount('/content/drive')


DATA_PATH = "/content/drive/MyDrive/ML/train_data"


Mounted at /content/drive


In [ ]:
df = pd.read_csv(DATA_PATH)
print("Rows, cols:", df.shape)
display(df.head())
print("\nColumn names:\n", df.columns.tolist())
# If label column is named differently, print unique values of the label column
if 'label' in df.columns:
    print("\nLabel distribution:\n", df['label'].value_counts().head(20))
else:
    print("No 'label' column found — adjust accordingly.")


Rows, cols: (25192, 42)


,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,class
0,0,tcp,ftp_data,SF,491,0,0,0,0,0,...,25,0.17,0.03,0.17,0.00,0.00,0.00,0.05,0.00,normal
1,0,udp,other,SF,146,0,0,0,0,0,...,1,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,normal
2,0,tcp,private,S0,0,0,0,0,0,0,...,26,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,anomaly
3,0,tcp,http,SF,232,8153,0,0,0,0,...,255,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,normal
4,0,tcp,http,SF,199,420,0,0,0,0,...,255,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,normal



Column names:
 ['duration', 'protocol_type', 'service', 'flag', 'src_bytes', 'dst_bytes', 'land', 'wrong_fragment', 'urgent', 'hot', 'num_failed_logins', 'logged_in', 'num_compromised', 'root_shell', 'su_attempted', 'num_root', 'num_file_creations', 'num_shells', 'num_access_files', 'num_outbound_cmds', 'is_host_login', 'is_guest_login', 'count', 'srv_count', 'serror_rate', 'srv_serror_rate', 'rerror_rate', 'srv_rerror_rate', 'same_srv_rate', 'diff_srv_rate', 'srv_diff_host_rate', 'dst_host_count', 'dst_host_srv_count', 'dst_host_same_srv_rate', 'dst_host_diff_srv_rate', 'dst_host_same_src_port_rate', 'dst_host_srv_diff_host_rate', 'dst_host_serror_rate', 'dst_host_srv_serror_rate', 'dst_host_rerror_rate', 'dst_host_srv_rerror_rate', 'class']
No 'label' column found — adjust accordingly.


In [ ]:
df['is_attack'] = (df['class'] != 'normal').astype(int)  # 1 = anomoly, 0 = normal
print(df['is_attack'].value_counts(normalize=False))

is_attack
0    13449
1    11743
Name: count, dtype: int64


In [ ]:
# Categorical features
cat_features = ['protocol_type', 'service', 'flag']

# Numerical features (all others except 'class' and the 3 categorical)
num_features = [
    'duration', 'src_bytes', 'dst_bytes', 'land', 'wrong_fragment', 'urgent',
    'hot', 'num_failed_logins', 'logged_in', 'num_compromised', 'root_shell',
    'su_attempted', 'num_root', 'num_file_creations', 'num_shells',
    'num_access_files', 'num_outbound_cmds', 'is_host_login', 'is_guest_login',
    'count', 'srv_count', 'serror_rate', 'srv_serror_rate', 'rerror_rate',
    'srv_rerror_rate', 'same_srv_rate', 'diff_srv_rate', 'srv_diff_host_rate',
    'dst_host_count', 'dst_host_srv_count', 'dst_host_same_srv_rate',
    'dst_host_diff_srv_rate', 'dst_host_same_src_port_rate',
    'dst_host_srv_diff_host_rate', 'dst_host_serror_rate',
    'dst_host_srv_serror_rate', 'dst_host_rerror_rate',
    'dst_host_srv_rerror_rate'
]

#Taking a sample of the dataset
df_sample = df.sample(n=3000, random_state=42).reset_index(drop=True)
selected_features = [
    'protocol_type', 'service', 'flag',
    'src_bytes', 'dst_bytes', 'count',
    'srv_count', 'same_srv_rate', 'dst_host_srv_count','class'
]

# Keep only those columns that actually exist in your dataset
selected_features = [f for f in selected_features if f in df_sample.columns]

# Step 3: Create the reduced dataframe
df_sample = df_sample[selected_features]

cat_features = [c for c in cat_features if c in df_sample.columns]
num_features = [c for c in num_features if c in df_sample.columns]

print("Categorical Features:", cat_features)
print("Number of Numerical Features:", len(num_features))

Categorical Features: ['protocol_type', 'service', 'flag']
Number of Numerical Features: 6


In [ ]:
#discretize numeric features for apriori (quantile bins)
n_bins = 5
kbd = KBinsDiscretizer(n_bins=n_bins, encode='ordinal', strategy='quantile')

# Fit-transform only on numeric cols and create string bins like 'src_bytes_bin=2'
df_bins = df_sample.copy()
if num_features:
    df_bins[num_features] = kbd.fit_transform(df_sample[num_features]).astype(int)
    for f in num_features:
        df_bins[f] = df_bins[f].astype(str).apply(lambda x: f + "_bin=" + x)          #giving name to bins created
print("Example discretized columns:")
display(df_bins[num_features].head())

'''
for displaying the edge values of the bins
for i, col in enumerate(num_features):
    print(f"{col} bins: {kbd.bin_edges_[i]}")
'''

Example discretized columns:


,src_bytes,dst_bytes,count,srv_count,same_srv_rate,dst_host_srv_count
0,src_bytes_bin=2,dst_bytes_bin=0,count_bin=1,srv_count_bin=3,same_srv_rate_bin=1,dst_host_srv_count_bin=3
1,src_bytes_bin=0,dst_bytes_bin=0,count_bin=1,srv_count_bin=3,same_srv_rate_bin=1,dst_host_srv_count_bin=2
2,src_bytes_bin=0,dst_bytes_bin=0,count_bin=2,srv_count_bin=3,same_srv_rate_bin=1,dst_host_srv_count_bin=0
3,src_bytes_bin=0,dst_bytes_bin=0,count_bin=0,srv_count_bin=2,same_srv_rate_bin=1,dst_host_srv_count_bin=1
4,src_bytes_bin=0,dst_bytes_bin=0,count_bin=1,srv_count_bin=3,same_srv_rate_bin=1,dst_host_srv_count_bin=1


'\nfor displaying the edge values of the bins\nfor i, col in enumerate(num_features):\n    print(f"{col} bins: {kbd.bin_edges_[i]}")\n'

In [ ]:
# Build transactional dataframe: combine categorical raw values with discretized numeric-bin strings
apriori_cols = []
for c in cat_features:
    # convert to strings like 'protocol_type=tcp'
    df_bins[c] = df_bins[c].astype(str).apply(lambda x: f"{c}={x}")
    apriori_cols.append(c)
apriori_cols += num_features

# Create a DataFrame where each cell is the token string for that feature
# then convert to one-hot via get_dummies on a stacked series — robust approach
trans = df_bins[apriori_cols].apply(lambda col: col.astype(str))
stacked = trans.stack()
one_hot = pd.get_dummies(stacked, prefix=None).groupby(level=0).sum()

pd.set_option('display.max_columns', None)
print("Transactional one-hot shape:", one_hot.shape)
one_hot.head()


Transactional one-hot shape: (3000, 97)


,count_bin=0,count_bin=1,count_bin=2,count_bin=3,dst_bytes_bin=0,dst_bytes_bin=1,dst_bytes_bin=2,dst_host_srv_count_bin=0,dst_host_srv_count_bin=1,dst_host_srv_count_bin=2,dst_host_srv_count_bin=3,flag=REJ,flag=RSTO,flag=RSTOS0,flag=RSTR,flag=S0,flag=S1,flag=S2,flag=S3,flag=SF,flag=SH,protocol_type=icmp,protocol_type=tcp,protocol_type=udp,same_srv_rate_bin=0,same_srv_rate_bin=1,service=IRC,service=X11,service=Z39_50,service=auth,service=bgp,service=courier,service=csnet_ns,service=ctf,service=daytime,service=discard,service=domain,service=domain_u,service=echo,service=eco_i,service=ecr_i,service=efs,service=exec,service=finger,service=ftp,service=ftp_data,service=gopher,service=hostnames,service=http,service=http_443,service=http_8001,service=imap4,service=iso_tsap,service=klogin,service=kshell,service=ldap,service=link,service=login,service=mtp,service=name,service=netbios_dgm,service=netbios_ns,service=netbios_ssn,service=netstat,service=nnsp,service=nntp,service=ntp_u,service=other,service=pop_2,service=pop_3,service=private,service=red_i,service=remote_job,service=rje,service=shell,service=smtp,service=sql_net,service=ssh,service=sunrpc,service=supdup,service=systat,service=telnet,service=time,service=urh_i,service=urp_i,service=uucp,service=uucp_path,service=vmnet,service=whois,src_bytes_bin=0,src_bytes_bin=1,src_bytes_bin=2,srv_count_bin=0,srv_count_bin=1,srv_count_bin=2,srv_count_bin=3,srv_count_bin=4
0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0
1,0,1,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0
2,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0
3,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0
4,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0


In [ ]:
# Apriori: find frequent itemsets
min_support = 0.04   # 0.01-0.05 depending on dataset size
frequent_itemsets = apriori(one_hot, min_support=min_support, use_colnames=True)
print("Frequent itemsets found:", len(frequent_itemsets))
display(frequent_itemsets.sort_values('support', ascending=False).head())

# Generate association rules
min_confidence = 0.6
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=min_confidence)
print("Total rules:", len(rules))
rules = rules.sort_values(['confidence','lift','support'], ascending=[False,False,False])
display(rules.head(10))


Frequent itemsets found: 2389


,support,itemsets
15,0.823667,(protocol_type=tcp)
18,0.814000,(same_srv_rate_bin=1)
202,0.638667,"(same_srv_rate_bin=1, protocol_type=tcp)"
24,0.600000,(src_bytes_bin=0)
4,0.600000,(dst_bytes_bin=0)


Total rules: 14580


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
4253,(service=domain_u),"(same_srv_rate_bin=1, protocol_type=udp, src_b...",0.066333,0.106667,0.066333,1.0,9.375,1.0,0.059258,inf,0.956801,0.621875,1.0,0.810937
8997,"(service=domain_u, flag=SF)","(protocol_type=udp, same_srv_rate_bin=1, src_b...",0.066333,0.106667,0.066333,1.0,9.375,1.0,0.059258,inf,0.956801,0.621875,1.0,0.810937
9000,(service=domain_u),"(flag=SF, protocol_type=udp, same_srv_rate_bin...",0.066333,0.106667,0.066333,1.0,9.375,1.0,0.059258,inf,0.956801,0.621875,1.0,0.810937
8685,"(service=domain_u, dst_host_srv_count_bin=3)","(protocol_type=udp, same_srv_rate_bin=1, src_b...",0.057333,0.106667,0.057333,1.0,9.375,1.0,0.051218,inf,0.947666,0.537500,1.0,0.768750
12634,"(service=domain_u, flag=SF, dst_host_srv_count...","(protocol_type=udp, same_srv_rate_bin=1, src_b...",0.057333,0.106667,0.057333,1.0,9.375,1.0,0.051218,inf,0.947666,0.537500,1.0,0.768750
12644,"(service=domain_u, dst_host_srv_count_bin=3)","(flag=SF, protocol_type=udp, same_srv_rate_bin...",0.057333,0.106667,0.057333,1.0,9.375,1.0,0.051218,inf,0.947666,0.537500,1.0,0.768750
9127,"(srv_count_bin=4, service=domain_u)","(protocol_type=udp, same_srv_rate_bin=1, src_b...",0.055333,0.106667,0.055333,1.0,9.375,1.0,0.049431,inf,0.945660,0.518750,1.0,0.759375
13021,"(srv_count_bin=4, flag=SF, service=domain_u)","(protocol_type=udp, same_srv_rate_bin=1, src_b...",0.055333,0.106667,0.055333,1.0,9.375,1.0,0.049431,inf,0.945660,0.518750,1.0,0.759375
13033,"(srv_count_bin=4, service=domain_u)","(flag=SF, protocol_type=udp, same_srv_rate_bin...",0.055333,0.106667,0.055333,1.0,9.375,1.0,0.049431,inf,0.945660,0.518750,1.0,0.759375
12928,"(srv_count_bin=4, dst_host_srv_count_bin=3, se...","(protocol_type=udp, same_srv_rate_bin=1, src_b...",0.052667,0.106667,0.052667,1.0,9.375,1.0,0.047049,inf,0.942998,0.493750,1.0,0.746875


In [ ]:
# Ensure the class/label column exists in your original df
attack_mask = df['class'] == 'anomaly'
normal_mask = df['class'] == 'normal'

rules_filtered = []
for idx, r in rules.iterrows():
    antecedents = list(r['antecedents'])
    if not set(antecedents).issubset(one_hot.columns):
        continue
    match_vec = one_hot[antecedents].all(axis=1)
    supp_attack = match_vec[attack_mask].mean()
    supp_normal = match_vec[normal_mask].mean()

    # much looser thresholds
    if supp_attack > 0.05 and (supp_attack / (supp_normal + 1e-9)) > 1.2:
        rules_filtered.append({
            'antecedents': antecedents,
            'support_attack': supp_attack,
            'support_normal': supp_normal,
            'confidence': r['confidence'],
            'lift': r['lift'],
            'support': r['support']
        })

print("Rules filtered (likely attack-indicative):", len(rules_filtered))



Rules filtered (likely attack-indicative): 780


In [ ]:
for i, rf in enumerate(rules_filtered[:10]):
    print(f"\nRule {i+1}: {rf['antecedents']}")
    print(f" Support (attack): {rf['support_attack']:.3f}")
    print(f" Support (normal): {rf['support_normal']:.3f}")
    print(f" Confidence: {rf['confidence']:.3f}, Lift: {rf['lift']:.3f}")



Rule 1: ['flag=SF', 'srv_count_bin=1', 'protocol_type=tcp']
 Support (attack): 0.118
 Support (normal): 0.098
 Confidence: 1.000, Lift: 2.566

Rule 2: ['flag=SF', 'srv_count_bin=1', 'same_srv_rate_bin=1', 'protocol_type=tcp']
 Support (attack): 0.118
 Support (normal): 0.098
 Confidence: 1.000, Lift: 2.566

Rule 3: ['flag=SF', 'srv_count_bin=1', 'protocol_type=tcp']
 Support (attack): 0.118
 Support (normal): 0.098
 Confidence: 1.000, Lift: 2.566

Rule 4: ['src_bytes_bin=2', 'srv_count_bin=1']
 Support (attack): 0.067
 Support (normal): 0.053
 Confidence: 1.000, Lift: 2.566

Rule 5: ['src_bytes_bin=2', 'same_srv_rate_bin=1', 'srv_count_bin=1']
 Support (attack): 0.067
 Support (normal): 0.053
 Confidence: 1.000, Lift: 2.566

Rule 6: ['src_bytes_bin=2', 'srv_count_bin=1']
 Support (attack): 0.067
 Support (normal): 0.053
 Confidence: 1.000, Lift: 2.566

Rule 7: ['flag=SF', 'src_bytes_bin=2', 'srv_count_bin=1']
 Support (attack): 0.066
 Support (normal): 0.052
 Confidence: 1.000, Lift: 

In [ ]:
#Build rule-based feature matrix
import numpy as np

print("Building rule-based features... this may take a minute depending on rule count.")
rule_features = pd.DataFrame(index=df.index)

# For each rule, check which rows satisfy its antecedents
for i, rf_rule in enumerate(rules_filtered):
    antecedents = rf_rule['antecedents']
    if not set(antecedents).issubset(one_hot.columns):
        continue
    match_vec = one_hot[antecedents].all(axis=1).astype(int)
    rule_features[f"rule_{i}"] = match_vec

print("Rule-based feature matrix shape:", rule_features.shape)
display(rule_features.head())


Building rule-based features... this may take a minute depending on rule count.
Rule-based feature matrix shape: (25192, 780)


,rule_0,rule_1,rule_2,rule_3,rule_4,rule_5,rule_6,rule_7,rule_8,rule_9,rule_10,rule_11,rule_12,rule_13,rule_14,rule_15,rule_16,rule_17,rule_18,rule_19,rule_20,rule_21,rule_22,rule_23,rule_24,rule_25,rule_26,rule_27,rule_28,rule_29,rule_30,rule_31,rule_32,rule_33,rule_34,rule_35,rule_36,rule_37,rule_38,rule_39,rule_40,rule_41,rule_42,rule_43,rule_44,rule_45,rule_46,rule_47,rule_48,rule_49,rule_50,rule_51,rule_52,rule_53,rule_54,rule_55,rule_56,rule_57,rule_58,rule_59,rule_60,rule_61,rule_62,rule_63,rule_64,rule_65,rule_66,rule_67,rule_68,rule_69,rule_70,rule_71,rule_72,rule_73,rule_74,rule_75,rule_76,rule_77,rule_78,rule_79,rule_80,rule_81,rule_82,rule_83,rule_84,rule_85,rule_86,rule_87,rule_88,rule_89,rule_90,rule_91,rule_92,rule_93,rule_94,rule_95,rule_96,rule_97,rule_98,rule_99,rule_100,rule_101,rule_102,rule_103,rule_104,rule_105,rule_106,rule_107,rule_108,rule_109,rule_110,rule_111,rule_112,rule_113,rule_114,rule_115,rule_116,rule_117,rule_118,rule_119,rule_120,rule_121,rule_122,rule_123,rule_124,rule_125,rule_126,rule_127,rule_128,rule_129,rule_130,rule_131,rule_132,rule_133,rule_134,rule_135,rule_136,rule_137,rule_138,rule_139,rule_140,rule_141,rule_142,rule_143,rule_144,rule_145,rule_146,rule_147,rule_148,rule_149,rule_150,rule_151,rule_152,rule_153,rule_154,rule_155,rule_156,rule_157,rule_158,rule_159,rule_160,rule_161,rule_162,rule_163,rule_164,rule_165,rule_166,rule_167,rule_168,rule_169,rule_170,rule_171,rule_172,rule_173,rule_174,rule_175,rule_176,rule_177,rule_178,rule_179,rule_180,rule_181,rule_182,rule_183,rule_184,rule_185,rule_186,rule_187,rule_188,rule_189,rule_190,rule_191,rule_192,rule_193,rule_194,rule_195,rule_196,rule_197,rule_198,rule_199,rule_200,rule_201,rule_202,rule_203,rule_204,rule_205,rule_206,rule_207,rule_208,rule_209,rule_210,rule_211,rule_212,rule_213,rule_214,rule_215,rule_216,rule_217,rule_218,rule_219,rule_220,rule_221,rule_222,rule_223,rule_224,rule_225,rule_226,rule_227,rule_228,rule_229,rule_230,rule_231,rule_232,rule_233,rule_234,rule_235,rule_236,rule_237,rule_238,rule_239,rule_240,rule_241,rule_242,rule_243,rule_244,rule_245,rule_246,rule_247,rule_248,rule_249,rule_250,rule_251,rule_252,rule_253,rule_254,rule_255,rule_256,rule_257,rule_258,rule_259,rule_260,rule_261,rule_262,rule_263,rule_264,rule_265,rule_266,rule_267,rule_268,rule_269,rule_270,rule_271,rule_272,rule_273,rule_274,rule_275,rule_276,rule_277,rule_278,rule_279,rule_280,rule_281,rule_282,rule_283,rule_284,rule_285,rule_286,rule_287,rule_288,rule_289,rule_290,rule_291,rule_292,rule_293,rule_294,rule_295,rule_296,rule_297,rule_298,rule_299,rule_300,rule_301,rule_302,rule_303,rule_304,rule_305,rule_306,rule_307,rule_308,rule_309,rule_310,rule_311,rule_312,rule_313,rule_314,rule_315,rule_316,rule_317,rule_318,rule_319,rule_320,rule_321,rule_322,rule_323,rule_324,rule_325,rule_326,rule_327,rule_328,rule_329,rule_330,rule_331,rule_332,rule_333,rule_334,rule_335,rule_336,rule_337,rule_338,rule_339,rule_340,rule_341,rule_342,rule_343,rule_344,rule_345,rule_346,rule_347,rule_348,rule_349,rule_350,rule_351,rule_352,rule_353,rule_354,rule_355,rule_356,rule_357,rule_358,rule_359,rule_360,rule_361,rule_362,rule_363,rule_364,rule_365,rule_366,rule_367,rule_368,rule_369,rule_370,rule_371,rule_372,rule_373,rule_374,rule_375,rule_376,rule_377,rule_378,rule_379,rule_380,rule_381,rule_382,rule_383,rule_384,rule_385,rule_386,rule_387,rule_388,rule_389,rule_390,rule_391,rule_392,rule_393,rule_394,rule_395,rule_396,rule_397,rule_398,rule_399,rule_400,rule_401,rule_402,rule_403,rule_404,rule_405,rule_406,rule_407,rule_408,rule_409,rule_410,rule_411,rule_412,rule_413,rule_414,rule_415,rule_416,rule_417,rule_418,rule_419,rule_420,rule_421,rule_422,rule_423,rule_424,rule_425,rule_426,rule_427,rule_428,rule_429,rule_430,rule_431,rule_432,rule_433,rule_434,rule_435,rule_436,rule_437,rule_438,rule_439,rule_440,rule_441,rule_442,rule_443,rule_444,rule_445,rule_446,rule_447,rule_448,rule_449,rule_450,rule_451,rule_452,rule_453,rule_454,rule_455,rule_

In [ ]:
# Choose small set of key numeric features to combine
selected_numeric = ['duration', 'src_bytes', 'dst_bytes', 'count', 'srv_count']

# Make sure they exist
selected_numeric = [f for f in selected_numeric if f in df.columns]

X_combined = pd.concat([df[selected_numeric].reset_index(drop=True),
                        rule_features.reset_index(drop=True)], axis=1)

y = df['is_attack'].astype(int).reset_index(drop=True)

print("Final feature matrix shape:", X_combined.shape)


Final feature matrix shape: (25192, 785)


In [ ]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_combined, y, test_size=0.3, random_state=42, stratify=y)

# Train RF
rf = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
rf.fit(X_train, y_train)

# Evaluate
y_pred = rf.predict(X_test)
print(classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.96      0.99      0.98      4035
           1       0.99      0.96      0.97      3523

    accuracy                           0.97      7558
   macro avg       0.98      0.97      0.97      7558
weighted avg       0.97      0.97      0.97      7558

Confusion Matrix:
 [[3993   42]
 [ 151 3372]]


In [ ]:
# Save RF model and the rule list for future inference
joblib.dump(rf, "rf_apriori_model.joblib")
joblib.dump(kbd, "kbd_discretizer.joblib")
joblib.dump(X_combined.columns.tolist(), "model_features.joblib")

# Save rules metadata
import json
with open("apriori_rules.json", "w") as f:
    json.dump(rules_filtered, f, indent=2)
print("Saved rf_apriori_model.joblib and apriori_rules.json")


Saved rf_apriori_model.joblib and apriori_rules.json


In [ ]:
''' Thoda sa wrong hai yeh predict function
import pandas as pd
import joblib
import json

# Load model and rules
rf = joblib.load("rf_apriori_model.joblib")
with open("apriori_rules.json", "r") as f:
    rules_filtered = json.load(f)

def detect_intrusion(new_record):
    """
    Takes a single input record (dict), applies one-hot encoding,
    aligns it with the model's expected features, and predicts if it's attack or normal.
    """
    # Convert dict to DataFrame
    new_df = pd.DataFrame([new_record])

    # One-hot encode categorical columns
    categorical_cols = ["protocol_type", "service", "flag"]
    new_encoded = pd.get_dummies(new_df, columns=categorical_cols)

    # Match columns with model training features
    model_features = rf.feature_names_in_
    for col in model_features:
        if col not in new_encoded.columns:
            new_encoded[col] = 0
    new_encoded = new_encoded[model_features]

    # Predict
    prediction = rf.predict(new_encoded)[0]
    prob = rf.predict_proba(new_encoded)[0][1]

    # Output
    print("Prediction:", "Attack Detected" if prediction == 1 else "Normal Connection")
    print(f"Attack Probability: {prob:.2f}")

'''

In [ ]:
# correct prediction function (need to run and check)
import pandas as pd
import joblib
import json

# Load artifacts
rf = joblib.load("rf_apriori_model.joblib")
kbd = joblib.load("kbd_discretizer.joblib")
model_features = joblib.load("model_features.joblib")
with open("apriori_rules.json","r") as f:
    rules_filtered = json.load(f)

CAT_COLS = ["protocol_type", "service", "flag"]
NUMERIC_COLS = [
    "src_bytes",
    "dst_bytes",
    "count",
    "srv_count",
    "same_srv_rate",
    "dst_host_srv_count"
]


def detect_intrusion(new_record):
    # 1) Convert to DataFrame
    df_row = pd.DataFrame([new_record])

    # 2) Discretize numeric values (same as training)
    num_present = [c for c in NUMERIC_COLS if c in df_row.columns]
    num_arr = df_row[num_present].astype(float).values.reshape(1, -1)
    bins = kbd.transform(num_arr).astype(int)[0]

    # Make token strings for bins
    tokens = []
    for i, col in enumerate(num_present):
        tokens.append(f"{col}_bin={bins[i]}")

    # 3) Add categorical tokens
    for c in CAT_COLS:
        if c in df_row.columns:
            tokens.append(f"{c}={df_row[c].iloc[0]}")

    # 4) Build rule features
    rule_values = {}
    token_set = set(tokens)

    for i, rule in enumerate(rules_filtered):
        antecedents = rule["antecedents"]
        matched = all(a in token_set for a in antecedents)
        rule_values[f"rule_{i}"] = int(matched)

    rule_df = pd.DataFrame([rule_values])

    # 5) Combine numeric + rule features
    numeric_df = pd.DataFrame(columns=NUMERIC_COLS)
    for c in NUMERIC_COLS:
        numeric_df.loc[0, c] = df_row[c].iloc[0] if c in df_row.columns else 0

    combined = pd.concat([numeric_df, rule_df], axis=1)

    # 6) Align to model_features
    aligned = pd.DataFrame(columns=model_features)
    for col in model_features:
        aligned.loc[0, col] = combined[col].iloc[0] if col in combined.columns else 0

    aligned = aligned.fillna(0).astype(float)

    # 7) Predict
    pred = int(rf.predict(aligned)[0])
    prob = float(rf.predict_proba(aligned)[0][1])

    print("Prediction:", "Attack" if pred == 1 else "Normal")
    print(f"Attack Probability: {prob:.3f}")

    return pred, prob


In [ ]:
sample = {
    "duration": 5,
    "protocol_type": "tcp",
    "service": "http",
    "flag": "SF",
    "src_bytes": 200,
    "dst_bytes": 1500,
    "count": 10,
    "srv_count": 5,
    "same_srv_rate": 0.8,
    "dst_host_srv_count": 15
}

detect_intrusion(sample)


Prediction: Normal
Attack Probability: 0.010


(0, 0.01)

In [ ]:
kbd.n_features_in_
kbd.feature_names_in_



array(['src_bytes', 'dst_bytes', 'count', 'srv_count', 'same_srv_rate',
       'dst_host_srv_count'], dtype=object)

In [ ]:
sample_attack = {
    'duration': 0,
    'protocol_type': 'tcp',
    'service': 'http',
    'flag': 'S0',     # suspicious: many SYNs with no reply
    'src_bytes': 0,
    'dst_bytes': 0,
    'count': 200,     # many connections in the window
    'srv_count': 180  # many to same service
}

detect_intrusion(sample_attack)


ValueError: X has 4 features, but KBinsDiscretizer is expecting 6 features as input.

In [ ]:
import os

files = os.listdir()
print(files)


['.config', 'kbd_discretizer.joblib', 'model_features.joblib', 'rf_apriori_model.joblib', 'drive', 'apriori_rules.json', 'sample_data']


In [ ]:
from google.colab import files
files.download("rf_apriori_model.joblib")
files.download("apriori_rules.json")
files.download("kbd_discretizer.joblib")
files.download("model_features.joblib")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>